In [1]:
# [swe_agent/agent/inputs.py]
import re
import typing as t
from pathlib import Path
from composio import Action, ComposioToolSet


InputType = t.TypeVar("InputType")


def read_user_input(
    prompt: str,
    metavar: str,
    validator: t.Callable[[str], InputType],
) -> InputType:
    """Read user input."""
    while True:
        value = input(f"{prompt} > ")
        try:
            return validator(value)
        except Exception as e:
            print(f"Invalid value for `{metavar}` error parsing `{value}`; {e}")


def _github_repository_name_validator(name: str) -> t.Tuple[str, str]:
    """Validate github repository name."""
    if " " in name:
        raise ValueError()
    owner, name = name.split("/")
    return owner, name


def _create_github_issue_validator(owner: str, name: str) -> t.Callable[[str], str]:
    """Create a github issue validator."""

    def _github_issue_validator(value: str) -> str:
        """Validate github issue."""
        # Check if it's a file path
        if Path(value).resolve().exists():
            return Path(value).read_text(encoding="utf-8")

        if re.match(r"^\d+$", value):
            response_data = ComposioToolSet().execute_action(
                action=Action.GITHUB_GET_AN_ISSUE,
                params={
                    "owner": owner,
                    "repo": name,
                    "issue_number": int(value),
                },
            ).get("data")
            if response_data:
                return response_data["body"]
            else:
                raise ValueError("No issue found with the given issue number")

        # If neither a file nor issue ID, treat as direct description text
        return value

    return _github_issue_validator


def from_github() -> t.Tuple[str, str]:
    """Take input from github."""
    owner, name = read_user_input(
        prompt="Enter github repository name",
        metavar="github repository name",
        validator=_github_repository_name_validator,
    )
    return (
        f"{owner}/{name}",
        read_user_input(
            prompt="Enter github issue ID or description or path to the file containing description",
            metavar="github issue",
            validator=_create_github_issue_validator(
                owner=owner,
                name=name,
            ),
        ),
    )

In [30]:
# read_user_input("Enter github repository name", "github repository name", _github_repository_name_validator)
# owner, name = _create_github_issue_validator("ElGreKost", "SoftwareDeveloperAgents")  # ElGreKost/SoftwareDeveloperAgents
# out = read_user_input("Enter github issue ID or description or path to the file containing description", "github issue", _create_github_issue_validator("ElGreKost", "SoftwareDeveloperAgents"))  # 1

In [2]:
import uuid
from agent import get_crew
from composio import Action
from tools import create_pr


In [3]:
repo, issue = from_github()
owner, repo_name = repo.split("/")

In [ ]:

crew, composio_toolset = get_crew(repo_path=f"/home/user/{repo_name}", workspace_id=None)
crew.kickoff(
    inputs={
        "repo": repo,
        "issue": issue,
    }
)
composio_toolset.execute_action(
    action=Action.FILETOOL_CHANGE_WORKING_DIRECTORY,
    params={"path": f"/home/user/{repo_name}"},
)
response = composio_toolset.execute_action(
    action=Action.FILETOOL_GIT_PATCH,
    params={},
)

In [5]:
from pathlib import Path
Path.home()

WindowsPath('C:/Users/kosti')